In [ ]:
# https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/fundamentals-raster-data/open-lidar-raster-python-xarray/

In [1]:
import xarray as xr
import netCDF4 as nc4
import rioxarray
import pandas as pd
import numpy as np
import datetime

/Users/Sun-jooLee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/Sun-jooLee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/Sun-jooLee/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
min_lon = 92.3032344909 
min_lat = 9.93295990645 
max_lon = 101.180005324 
max_lat = 28.335945136 

In [3]:
nc = nc4.Dataset('fldas.nc', mode='r')  
dataset = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))

In [4]:
fldas = dataset.sel(Y=slice(min_lat,max_lat), X=slice(min_lon,max_lon))

In [5]:
fldas = fldas.drop_vars('time_bnds').expand_dims({'source':['fldas']}).rename({'X':'lon', 'Y':'lat'})

In [6]:
fldas

<xarray.Dataset>
Dimensions:                 (source: 1, time: 1, lon: 89, lat: 184)
Coordinates:
  * source                  (source) object 'fldas'
  * time                    (time) datetime64[ns] 2017-01-01
  * lon                     (lon) float64 92.35 92.45 92.55 ... 101.1 101.2
  * lat                     (lat) float64 9.95 10.05 10.15 ... 28.05 28.15 28.25
Data variables: (12/28)
    Evap_tavg               (source, time, lat, lon) float32 nan ... 4.396e-06
    LWdown_f_tavg           (source, time, lat, lon) float32 nan nan ... 185.5
    Lwnet_tavg              (source, time, lat, lon) float32 nan nan ... -112.2
    Psurf_f_tavg            (source, time, lat, lon) float32 nan ... 6.505e+04
    Qair_f_tavg             (source, time, lat, lon) float32 nan ... 0.002086
    Qg_tavg                 (source, time, lat, lon) float32 nan nan ... -3.72
    ...                      ...
    SoilMoi40_100cm_tavg    (source, time, lat, lon) float32 nan nan ... 0.3508
    SoilMoi100_200cm_tavg   (source, time, lat, lon) float32 nan nan ... 0.3749
    SoilTemp00_10cm_tavg    (source, time, lat, lon) float32 nan nan ... 270.8
    SoilTemp10_40cm_tavg    (source, time, lat, lon) float32 nan nan ... 274.0
    SoilTemp40_100cm_tavg   (source, time, lat, lon) float32 nan nan ... 274.6
    SoilTemp100_200cm_tavg  (source, time, lat, lon) float32 nan nan ... 275.2
Attributes: (12/19)
    CDI:                       Climate Data Interface version 1.9.1 (http://m...
    history:                   Fri Oct 30 13:41:48 2020: cdo -z zip_4 replace...
    institution:               NASA GSFC
    Conventions:               CF-1.4
    missing_value:             -9999.0
    NUM_SOIL_LAYERS:           4
    ...                        ...
    SOUTH_WEST_CORNER_LON:     -179.95
    DX:                        0.1
    DY:                        0.1
    nco_openmp_thread_number:  1
    NCO:                       4.4.4
    CDO:                       Climate Data Operators version 1.9.1 (http://m...

In [7]:
da = rioxarray.open_rasterio("../data/geo/landscan/global/landscan-global-2017.tif")
lscn = da.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)

In [8]:
lscn = lscn.where(lscn != lscn.attrs['_FillValue'], np.nan)

In [9]:
lscn = lscn.drop('band').drop('spatial_ref')
lscn.name = 'landscan'
lscn = lscn.to_dataset()
lscn = lscn.expand_dims({'time':[datetime.date(2017,1,1)], 'source':['landscan']}).rename({'x':'lon', 'y':'lat'}).squeeze('band')

In [10]:
lscn

<xarray.Dataset>
Dimensions:   (time: 1, source: 1, lon: 1066, lat: 2210)
Coordinates:
  * time      (time) object 2017-01-01
  * source    (source) object 'landscan'
  * lon       (lon) float64 92.3 92.31 92.32 92.33 ... 101.2 101.2 101.2 101.2
  * lat       (lat) float64 28.34 28.33 28.32 28.31 ... 9.954 9.946 9.937 9.929
Data variables:
    landscan  (time, source, lat, lon) float64 1.0 1.0 0.0 0.0 ... nan nan nan

In [11]:
nc = nc4.Dataset("../data/geo/landcover/global/C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1-002.nc", mode='r')  # Or from siphon.ncss
lcvr = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))

lons = lcvr.lon.values
lats = lcvr.lat.values
mnln = min(range(len(lons)), key=lambda i: abs(lons[i]-min_lon))
mxln = min(range(len(lons)), key=lambda i: abs(lons[i]-max_lon))
mxlt = min(range(len(lats)), key=lambda i: abs(lats[i]-min_lat))
mnlt = min(range(len(lats)), key=lambda i: abs(lats[i]-max_lat))

lcvr = lcvr.isel(lat=slice(mnlt, mxlt), lon=slice(mnln, mxln))

In [12]:
lcvr = lcvr['lccs_class']
lcvr.name = 'landcover'
lcvr = lcvr.to_dataset()
lcvr = lcvr.expand_dims({'source':['landcover']})
lcvr.coords['time'].attrs = fldas.coords['time'].attrs

In [13]:
lcvr

<xarray.Dataset>
Dimensions:    (source: 1, lat: 6625, lon: 3195, time: 1)
Coordinates:
  * source     (source) object 'landcover'
  * lat        (lat) float64 28.33 28.33 28.33 28.33 ... 9.943 9.94 9.938 9.935
  * lon        (lon) float64 92.3 92.31 92.31 92.31 ... 101.2 101.2 101.2 101.2
  * time       (time) datetime64[ns] 2017-01-01
Data variables:
    landcover  (source, time, lat, lon) uint8 130 130 130 130 ... 210 210 210

In [14]:
da = rioxarray.open_rasterio("../data/geo/viirs/global/vcmcfg_avg_rade9h_201705.tif")
viirs = da.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)

In [15]:
viirs = viirs.drop('band').drop('spatial_ref')
viirs.name = 'viirs'
viirs = viirs.to_dataset()

In [16]:
viirs = viirs.expand_dims({'time':[datetime.date(2017,1,1)], 'source':['viirs']}).rename({'x':'lon', 'y':'lat'}).squeeze('band')

In [17]:
viirs

<xarray.Dataset>
Dimensions:  (time: 1, source: 1, lon: 2131, lat: 4418)
Coordinates:
  * time     (time) object 2017-01-01
  * source   (source) object 'viirs'
  * lon      (lon) float64 92.3 92.31 92.31 92.32 ... 101.2 101.2 101.2 101.2
  * lat      (lat) float64 28.34 28.33 28.33 28.32 ... 9.946 9.942 9.937 9.933
Data variables:
    viirs    (time, source, lat, lon) float32 0.3 0.36 0.35 ... 0.3 0.26 0.19